In [174]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import widgets
from ipywidgets import interactive
from IPython.display import Audio, display
import numpy as np

In [175]:
from colormath.color_objects import XYZColor, sRGBColor, SpectralColor
from colormath.color_conversions import convert_color

In [176]:
def get_spec_for_cm(wvl, amp):
    """Get a pandas series with index appropriate to colormath pectral color density with a single nonzero power"""
    assert 340 <= wvl <= 830, "we require 340 < wvl < 830"
    spec = pd.Series(index=range(340, 831, 10), data=0.0)
    spec[int(np.round(wvl / 10.) * 10.)] = amp  # round the signals to the nearest 10
    return spec

# do summation on the pandas series returned by the above function,
# then pass those series into the following functions:

def get_cm_sc_args(spec):
    spec_dict = spec[spec>0].to_dict()
    return {f"spec_{wvl}nm": spec_dict[wvl] for wvl in spec_dict}

def cmsc_from_spec(spec):
    return SpectralColor(**get_cm_sc_args(spec))

In [177]:
c_light_mps = 299792458.  # speed of light in meters per second
c_sound_mps = 343.  # speed of sound in meters per second

# conversion from wavelength to frequency:

# freq_light_Hz = c_light_mps / (wvl_light_m) = c_light_mps / (wvl_light_nM * 1e-9)

min_wvl_nM = 381. 
max_wvl_nM = 779. 
min_freq_EM_Hz = c_light_mps / (max_wvl_nM * 1e-9)
max_freq_EM_Hz = c_light_mps / (min_wvl_nM * 1e-9)

min_pitch_hz = 60.
max_wvl_sound_m = c_sound_mps / min_pitch_hz
max_pitch_hz = 4000.
min_wvl_sound_m = c_sound_mps / max_pitch_hz

# let's map light frequency to sound frequency linearly, such that the mins and maxes of the 
# respective scales match up:

# pitch_hz = m * EM_freq_hz + b

# min_pitch_hz = m * min_freq_EM_Hz + b
# max_pitch_hz = m * max_freq_EM_Hz + b
# solve these for m and b: 

# m = (max_pitch_hz - min_pitch_hz) / (max_freq_EM_Hz - min_freq_EM_Hz)
# b = min_pitch_hz - m * min_freq_EM_Hz

# def em_wvl_nm_to_em_hz(wvl_nm, c_light_mps = c_light_mps):
#     return c_light_mps / (wvl_nm * 1e-9)

# def em_hz_to_pitch_hz(em_hz, m=m, b=b):
#     return m * em_hz + b

# def em_wvl_nm_to_pitch_hz(wvl_nm, c_light_mps=c_light_mps, c_sound_mps=c_sound_mps, m=m, b=b):
#     em_hz = em_wvl_nm_to_em_hz(wvl_nm)
#     return em_hz_to_pitch_hz(em_hz)

In [178]:
# attempt 2: let's do the conversion between wavelengths, then map to sound frequency

In [179]:
m = (max_wvl_sound_m - min_wvl_sound_m) / (max_wvl_nM - min_wvl_nM)
b = min_wvl_sound_m - m * min_wvl_nM

In [180]:
def desired_sound_wvl_from_wvl_light(wvl_light, m=m, b=b):
    return m * wvl_light + b

def desired_pitch_from_wvl_light(wvl_light, m=m, b=b, c_sound_mps=c_sound_mps):
    desired_sound_wvl = desired_sound_wvl_from_wvl_light(wvl_light, m=m, b=b)
    desired_pitch = c_sound_mps / desired_sound_wvl
    return desired_pitch

In [181]:
# rate=8000
# max_time=5
# times = np.linspace(0, max_time, rate * 5)
# sig_test =  1*np.sin(2 * np.pi * 12024 * times)
# display(Audio(data=sig_test, rate=rate, autoplay=True, normalize=False))

In [182]:
from matplotlib_venn import venn3, venn3_circles
from itertools import product

def tritone_metamers_cm(wvl_1=460, wvl_2=530, wvl_3=610.60, a1=0.75, a2=0.85, a3=0.85):
    """autoplay sound and matplotlib display"""
    max_time = 15 # make sound file short to speed loading for interaction
    rate = 8000
    
    amp_tot = a1 + a2 + a3
    if amp_tot > 1.0:
        a1 /= amp_tot
        a2 /= amp_tot
        a3 /= amp_tot
    pitch_1 = desired_pitch_from_wvl_light(wvl_1)
    pitch_2 = desired_pitch_from_wvl_light(wvl_2)
    pitch_3 = desired_pitch_from_wvl_light(wvl_3)
    
    print(f"wvl_1 = {wvl_1} nM and pitch_1 = {pitch_1:.2f} Hz")
    print(f"wvl_2 = {wvl_2} nM and pitch_2 = {pitch_2:.2f} Hz")
    print(f"wvl_3 = {wvl_3} nM and pitch_3 = {pitch_3:.2f} Hz")
    
    times = np.linspace(0, max_time, rate * max_time)
    sig1 = a1 * np.sin(2 * np.pi * pitch_1 * times) 
    sig2 = a2 * np.sin(2 * np.pi * pitch_2 * times)
    sig3 = a3 * np.sin(2 * np.pi * pitch_3 * times)
    sig_tot = sig1 + sig2 + sig3

    display(Audio(data=sig_tot, rate=rate, autoplay=True, normalize=False))
    
    scale = 15
    spec1 = scale * get_spec_for_cm(wvl_1, a1)
    spec2 = scale * get_spec_for_cm(wvl_2, a2)
    spec3 = scale * get_spec_for_cm(wvl_3, a3)

    plt.figure(figsize=(4, 4), facecolor='k')
    v = venn3(subsets=(1, 1, 1, 1, 1, 1, 1), set_labels = ('', '', ''))

    # iterate over each patch. Patches in matplotlib_venn are identified by
    # strings like '010', which is the intersection of the second set with the complements of 1st and 3rd.
    # I think.
    for elem in product([0,1], [0,1], [0,1]):
        if elem != (0,0,0):
            this_id = str(elem[0]) + str(elem[1]) + str(elem[2])  # eg '100', '010', etc
            # the color in the patch is the sum of the signal components represented in the patch
            this_spec = elem[0]*spec1 + elem[1]*spec2 + elem[2]*spec3 
            this_cmsc = cmsc_from_spec(this_spec)
            this_rgb = convert_color(this_cmsc, sRGBColor)
            this_clamped_rgb = sRGBColor(this_rgb.clamped_rgb_r,
                                         this_rgb.clamped_rgb_g,
                                         this_rgb.clamped_rgb_b)

            v.get_patch_by_id(this_id).set_color(this_clamped_rgb.get_rgb_hex())
            v.get_patch_by_id(this_id).set_alpha(1.0)
            v.get_label_by_id(this_id).set_text('')

    c = venn3_circles(subsets=(1, 1, 1, 1, 1, 1, 1), linestyle='-')

    plt.show()
    return

## For the sliders, "wvl" stands for "wavelength", and "a" stands for "amplitude" or loudness/intensity

In [184]:
# autoplay interactive
v = interactive(tritone_metamers_cm, 
                wvl_1=(381.0,779.0, 2), 
                wvl_2=(381.0,779.0, 2), 
                wvl_3=(381.0,779.0, 2), 
                a1=(0.0,1.0, .05),
                a2=(0.0,1.0, .05), 
                a3=(0.0,1.0, .05))
display(v)

interactive(children=(FloatSlider(value=460.0, description='wvl_1', max=779.0, min=381.0, step=2.0), FloatSlid…

Wavelength to frequency: freq = c / wavelength, where c is the speed of propogation of the wave

In [125]:
desired_pitch_from_wvl_light(460)

266.4883002788656

In [28]:


EM_wvl_m_1 = wvl_1 * 1e-9
EM_wvl_m_2 = wvl_2 * 1e-9
EM_wvl_m_3 = wvl_3 * 1e-9

EM_freq_1 = c_light_mps / EM_wvl_m_1
EM_freq_2 = c_light_mps / EM_wvl_m_2
EM_freq_3 = c_light_mps / EM_wvl_m_3

pitch_freq_1 = m * EM_freq_1 + b
pitch_freq_2 = m * EM_freq_2 + b
pitch_freq_3 = m * EM_freq_3 + b




In [79]:
[em_wvl_nm_to_pitch_hz(wvl) for wvl in range(380, 800, 20)]

[12222.404809437383,
 10955.920775862065,
 9810.054269293923,
 8768.357445141064,
 7817.242953523237,
 6945.388002873562,
 6143.281448275859,
 5402.875397877986,
 4717.314240102172,
 4080.7217364532007,
 3488.0321640903676,
 2934.855229885059,
 2417.3671301446084,
 1932.2220366379333,
 1476.479676071056,
 1047.545689655175,
 643.1222167487686,
 261.1667145593874,
 -100.14254426840307,
 -442.4355263157886,
 -767.1750221043294]

In [173]:
[a**.1 for a in np.arange(0,1,0.05)]

[0.0,
 0.7411344491069477,
 0.7943282347242815,
 0.8271973337231157,
 0.8513399225207846,
 0.8705505632961241,
 0.8865681505652133,
 0.9003405372962772,
 0.9124435365554808,
 0.9232541136674268,
 0.9330329915368074,
 0.9419682592138262,
 0.9502002165056764,
 0.9578363965785066,
 0.9649610951198176,
 0.9716416578630735,
 0.9779327685429285,
 0.9838794565405263,
 0.9895192582062144,
 0.9948838031081763]